In [ ]:
import requests
import sys
sys.path.append('..')
from ltr.client.solr_client import SolrClient

client = SolrClient(host='http://aips-solr:8983/solr')

# Listing 10.3 

In [ ]:
from ltr.judgments import Judgment

Judgment(grade=1, keywords='social network', doc_id=37799)

# Listing 10.4

In [ ]:
mini_judg_list=[
    # for 'social network' query
    Judgment(grade=1, keywords='social network', doc_id='37799'), #The Social Network
    Judgment(grade=0, keywords='social network', doc_id='267752'), # #chicagoGirl
    Judgment(grade=0, keywords='social network', doc_id='38408'), # Life As We Know It
    Judgment(grade=0, keywords='social network', doc_id='28303'), # The Cheyenne Social Club
    
    # for 'star wars' query
    Judgment(grade=1, keywords='star wars', doc_id='11'), # star wars
    Judgment(grade=1, keywords='star wars', doc_id='1892'), # return of jedi
    Judgment(grade=0, keywords='star wars', doc_id='54138'),# Star Trek Into Darkness
    Judgment(grade=0, keywords='star wars', doc_id='85783'), # The Star
    Judgment(grade=0, keywords='star wars', doc_id='325553'), # Battlestar Galactica
]

mini_judg_list

# Listing 10.5

In [ ]:
mini_judg_list[0].features


## Listing 10.6

In [ ]:
requests.delete('http://aips-solr:8983/solr/tmdb/schema/feature-store/movies')

import requests

feature_set = [
    {
      "name" : "title_bm25",
      "store": "movies",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : { #q=title:({$keywords})
        "q" : "title:(${keywords})"
      }
    },
    {
      "name" : "overview_bm25",
      "store": "movies",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "overview:(${keywords})"
      }
    },
    {
      "name" : "release_year",
      "store": "movies",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "{!func}release_year"
}}]

resp = requests.put('http://aips-solr:8983/solr/tmdb/schema/feature-store',
                    json=feature_set)
resp.text

# Listing 10.07 & 10.08


In [ ]:
import requests

logging_solr_query = {
    "fl": "id,title,[features store=movies efi.keywords=\"social network\"]",
    'q': "id:37799 OR id:267752 id:38408 OR id:28303", #social network graded documents
    'rows': 10,
    'wt': 'json'  
}

resp = requests.post('http://aips-solr:8983/solr/tmdb/select',
                     data=logging_solr_query)

resp.json()


## Listing 10.09 and 10.10 output Generation Code
**The following code is omitted from the book, but the parsed output is included as listing 10.09**

This code just shows boring parsing of the Solr logging response for a single query (CURR_QID). The FeatureLogger class does all this for you (and helps account for bookkeeping). But we do want to show what this would look like if you recreated this in your own code

In [ ]:
def populate_features_for_qid(qid, solr_resp, judg_list):
    solr_json = solr_resp.json()
    doc_id_to_features = {}

    # Map Doc Id => Features
    for doc in solr_json['response']['docs']:
        # Parse '[features] array', ie
        # title_bm25=0.0,overview_bm25=13.237938,vote_average=7.0'
        features = doc['[features]']
        features = features.split(',')
        features = [float(ftr.split('=')[1]) for ftr in features]

        doc_id_to_features[doc['id']] = features

    # Save in correct judgment
    for judgment in judg_list:
        if judgment.qid == qid:
            try:
                judgment.features = doc_id_to_features[judgment.doc_id]
            except KeyError:
                pass

populate_features_for_qid(qid=1, solr_resp=resp, judg_list=mini_judg_list)
mini_judg_list

In [ ]:
logging_solr_query = {
    "fl": "id,title,[features store=movies efi.keywords=\"star wars\"]",
    'q': "id:11 OR id:1892 OR id:54138 OR id:85783 OR id:325553", #star wars graded documents
    'rows': 10,
    'wt': 'json'  
}

resp = requests.post('http://aips-solr:8983/solr/tmdb/select',
                     data=logging_solr_query)

populate_features_for_qid(qid=2, solr_resp=resp, judg_list=mini_judg_list)

mini_judg_list

## Loading / logging training set (omitted from book)

In [ ]:
from ltr.log import FeatureLogger
from ltr.judgments import judgments_open
from itertools import groupby
from ltr import download

judgments='http://es-learn-to-rank.labs.o19s.com/ai_pow_search_judgments.txt'
download([judgments], dest='data/')

ftr_logger=FeatureLogger(client, index='tmdb', feature_set='movies')

with judgments_open('data/ai_pow_search_judgments.txt') as judgment_list:
    for qid, query_judgments in groupby(judgment_list, key=lambda j: j.qid):
        ftr_logger.log_for_qid(judgments=query_judgments, 
                               qid=qid,
                               keywords=judgment_list.keywords(qid))
    
ftr_logger.logged